In [9]:
require 'loadcaffe'
require 'inn'
require 'image'
require 'dcnn'

-- Loads the mapping from net outputs to human readable labels
function load_synset()
  local file = io.open 'synset_words.txt'
  local list = {}
  while true do
    local line = file:read()
    if not line then break end
    table.insert(list, string.sub(line,11))
  end
  return list
end

-- Converts an image from RGB to BGR format and subtracts mean
function preprocess(im, img_mean)
  -- rescale the image
  local im3 = image.scale(im,224,224,'bilinear')*255
  -- RGB2BGR
  local im4 = im3:clone()
  im4[{1,{},{}}] = im3[{3,{},{}}]
  im4[{3,{},{}}] = im3[{1,{},{}}]

  -- subtract imagenet mean
  return im4 - image.scale(img_mean, 224, 224, 'bilinear')
end

-- Setting up networks 
print '==> Loading network'

prototxt_name = '/usr/local/data/CaffeZoo/VGG_ILSVRC_16_layers/VGG_ILSVRC_16_layers_deploy.prototxt'
binary_name = '/usr/local/data/CaffeZoo/VGG_ILSVRC_16_layers/VGG_ILSVRC_16_layers.caffemodel'

net = loadcaffe.load(prototxt_name, binary_name, 'cudnn')

net:cuda()
-- as we want to classify, let's disable dropouts by enabling evaluation mode
net:evaluate()

print '==> Loading synsets'
synset_words = load_synset()

print '==> Loading image and imagenet mean'
image_name = 'Goldfish3.jpg'
--image_name = 'lena.jpg'
--image_name='people.jpg'
img_mean_name = 'ilsvrc_2012_mean.t7'

im = image.load(image_name)
img_mean = torch.load(img_mean_name).img_mean:transpose(3,1)

-- Have to resize and convert from RGB to BGR and subtract mean
print '==> Preprocessing'
I = preprocess(im, img_mean)


-- Replace pooling by dual pooling
unpooling_layers = dcnn:ReplaceDualPoolingModule(net)

_,classes = net:forward(I:cuda()):view(-1):float():sort(true)

for i=1,5 do
  print('predicted class '..tostring(i)..': ', synset_words[classes[i] ])
end

==> Loading network	


Successfully loaded /usr/local/data/CaffeZoo/VGG_ILSVRC_16_layers/VGG_ILSVRC_16_layers.caffemodel


conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3


conv3_1: 256 128 3 3
conv3_2: 256 256 3 3


conv3_3: 256 256 3 3
conv4_1: 512 256 3 3


...

fc8: 1 1 4096 1000


==> Loading synsets	


==> Loading image and imagenet mean	


==> Preprocessing	


predicted class 1: 	goldfish, Carassius auratus	
predicted class 2: 	tench, Tinca tinca	
predicted class 3: 	gar, garfish, garpike, billfish, Lepisosteus osseus	
predicted class 4: 	coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch	
predicted class 5: 	coral reef	


In [ ]:
function dcnn:CreateDeconvNet( net, unpooling_layers, input_img, layer_num, neuron_index, normal_deconv  )
    neuron_index = neuron_index or 0
    if neuron_index == true then
        neuron_index = 0 
        normal_deconv = true
    end
    
    if neuron_index == false then
        neuron_index = 0 
        normal_deconv = false
    end 
    
    deconvNet = nn.Sequential()
    local first_deconv = true
    local unpooling_idx = 1
    for i=layer_num,1,-1 do
        if torch.typename( net:get(i)) == 'cudnn.SpatialConvolution' then
            local reconstructed_size = 0
            if i==1 then
                reconstructed_size = input_img:size(2)
            else    
                reconstructed_size = net:get(i-1).output:size(2)
            end
                
            if first_deconv == true then
                if neuron_index ~= 0 then 
                    deconvNet:add( cudnn.SpatialDeconvolution( net:get(i), reconstructed_size, neuron_index ))
                else
                    deconvNet:add( cudnn.SpatialDeconvolution( net:get(i), reconstructed_size, normal_deconv ))
                end
                
                first_deconv = false
            else
                deconvNet:add( cudnn.SpatialDeconvolution( net:get(i), reconstructed_size, true ))
            end
            
        elseif torch.typename( net:get(i)) == 'dcnn.DualSpatialMaxPooling' then
            local unpooling_idx = 1
            for j=i,1,-1 do  
                if torch.typename( net:get(j)) == 'dcnn.DualSpatialMaxPooling' then
                   unpooling_idx = unpooling_idx + 1
                end
            end
            
            deconvNet:add( unpooling_layers[unpooling_idx])
        
        elseif torch.typename( net:get(i)) == cudnn.ReLU then
            deconvNet:add( cudnn.ReLU(true) )
            
        else
            error('This type of layer is not supported !')
        end
    end    
        
    return deconvNet
end

In [10]:
net

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> (17) -> (18) -> (19) -> (20) -> (21) -> (22) -> (23) -> (24) -> (25) -> (26) -> (27) -> (28) -> (29) -> (30) -> (31) -> (32) -> (33) -> (34) -> (35) -> (36) -> (37) -> (38) -> (39) -> (40) -> output]
  (1): cudnn.SpatialConvolution(3 -> 64, 3x3, 1,1, 1,1)
  (2): cudnn.ReLU
  (3): cudnn.SpatialConvolution(64 -> 64, 3x3, 1,1, 1,1)
  (4): cudnn.ReLU
  (5): dcnn.DualSpatialMaxPooling(2,2,2,2)
  (6): cudnn.SpatialConvolution(64 -> 128, 3x3, 1,1, 1,1)
  (7): cudnn.ReLU
  (8): cudnn.SpatialConvolution(128 -> 128, 3x3, 1,1, 1,1)
  (9): cudnn.ReLU
  (10): dcnn.DualSpatialMaxPooling(2,2,2,2)
  (11): cudnn.SpatialConvolution(128 -> 256, 3x3, 1,1, 1,1)
  (12): cudnn.ReLU
  (13): cudnn.SpatialConvolution(256 -> 256, 3x3, 1,1, 1,1)
  (14): cudnn.ReLU
  (15): cudnn.SpatialConvolution(256 -> 256, 3x3, 1,1, 1,1)
  (16): cudnn.ReLU
  (17): dcnn.DualSpatialMax

          gradInput : CudaTensor - size: 3x224x224
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41995a50
          iSize : LongStorage - size: 4
          convDesc : cdata<struct cudnnConvolutionStruct *[1]>: 0x41e38870
          output_offset : 3211264
          gradBias : CudaTensor - size: 64
          algType : cdata<enum 761 [?]>: 0x40fa16c0
          extraBuffer : CudaTensor - empty
          extraBufferSizeInBytes : 0
          oDescForBias : cdata<struct cudnnTensorStruct *[1]>: 0x41a6cf70
          oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41e52f28
          groups : 1
          dH : 1
          dW : 1
          nOutputPlane : 64
          padH : 1
          kH : 3
          weight_offset : 1728
          finput : CudaTensor - empty
          biasDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4136ba98
          input_offset : 150528
          weight : CudaTensor - size: 64x3x3x3
          train : false
          gradWeight : CudaTensor - size: 64x3x3x3
     

mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 64x224x224
          gradInput : CudaTensor - empty
          nElem : 3211264
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40f9b638
        }
      3 : 
        cudnn.SpatialConvolution(64 -> 64, 3x3, 1,1, 1,1)
        {
          padW : 1
          nInputPlane : 64
          output : CudaTensor - size: 64x224x224
          gradInput : CudaTensor - size: 64x224x224
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x401dc778
          iSize : LongStorage - size: 4
          convDesc : cdata<struct cudnnConvolutionStruct *[1]>: 0x4136e4b8
          output_offset : 3211264
          gradBias : CudaTensor - size: 64
          algType : cdata<enum 761 [?]>: 0x401163e8
          extraBuffer : CudaTensor - empty
          extraBufferSizeInBytes : 0
          oDescForBias : cdata<struct cudnnTensorStruct *[1]>: 0x41b6c2d8
          oDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41678

      input_offset : 3211264
          weight : CudaTensor - size: 64x64x3x3
          train : false
          gradWeight : CudaTensor - size: 64x64x3x3
          fgradInput : CudaTensor - empty
          kW : 3
          bias : CudaTensor - size: 64
          weightDesc : cdata<struct cudnnFilterStruct *[1]>: 0x40513910
        }
      4 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 64x224x224
          gradInput : CudaTensor - empty
          nElem : 3211264
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4133ec20
        }
      5 : 
        dcnn.DualSpatialMaxPooling(2,2,2,2)
        {
          dH : 2
          dW : 2
          padH : 0
          gradInput : CudaTensor - empty
          indices : CudaTensor - size: 64x112x112
          dualModule : 
            dcnn.DualSpatialMaxUnpooling
            {
              gradInput : CudaTensor - empty
             

                dcnn.DualSpatialMaxPooling(2,2,2,2)
                {
                  dH : 2
                  dW : 2
                  padH : 0
                  gradInput : CudaTensor - empty
                  indices : CudaTensor - size: 64x112x112
                  dualModule : dcnn.DualSpatialMaxUnpooling
                  kH : 2
                  output : CudaTensor - size: 64x112x112
                  padW : 0
                  kW : 2
                }
              output : CudaTensor - size: 64x224x224
            }
          kH : 2
          output : CudaTensor - size: 64x112x112
          padW : 0
          kW : 2
        }
      6 : 
        cudnn.SpatialConvolution(64 -> 128, 3x3, 1,1, 1,1)
  

...

          biasDesc : cdata<struct cudnnTensorStruct *[1]>: 0x4147bad8
          input_offset : 100352
          weight : CudaTensor - size: 512x512x3x3
          train : false
          gradWeight : CudaTensor - size: 512x512x3x3
          fgradInput : CudaTensor - empty
          kW : 3
          bias : CudaTensor - size: 512
          weightDesc : cdata<struct cudnnFilterStruct *[1]>: 0x41a84008
        }
      30 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 512x14x14
          gradInput : CudaTensor - empty
          nElem : 100352
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x40115210
        }
      31 : 
        dcnn.DualSpatialMaxPooling(2,2,2,2)
        {
          dH : 2
          dW : 2
          padH : 0
          gradInput : CudaTensor - empty
          indices : CudaTensor - size: 512x7x7
          dualModule : 
            dcnn.DualSpatialMaxUnpooli

          dH : 2
                  dW : 2
                  padH : 0
                  gradInput : CudaTensor - empty
                  indices : CudaTensor - size: 512x7x7
                  dualModule : dcnn.DualSpatialMaxUnpooling
                  kH : 2
                  output : CudaTensor - size: 512x7x7
                  padW : 0
                  kW : 2
                }
              output : CudaTensor - size: 512x14x14
            }
          kH : 2
          output : CudaTensor - size: 512x7x7
          padW : 0
          kW : 2
        }
      32 : 
        nn.View
        {
          train : false
          output : CudaTensor - size: 25088
          numInputDims : 3
          size : LongStorage - size: 1
          numElements : 1
        }
      33 : 
        nn.Linear(25088 -> 4096)
        {
          gradBias : CudaTensor - size: 4096


          weight : CudaTensor - size: 4096x25088
          train : false
          bias : CudaTensor - size: 4096
          gradInput : CudaTensor - empty
          gradWeight : CudaTensor - size: 4096x25088
          output : CudaTensor - size: 4096
        }
      34 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 4096
          gradInput : CudaTensor - empty
          nElem : 4096
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41372488
        }
      35 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : CudaTensor - empty
          train : false
          p : 0.5
          gradInput : CudaTensor - empty
          output : CudaTensor - size: 4096
        }
      36 : 
        nn.Linear(4096 -> 4096)
        {
          gradBias : CudaTensor - size: 4096
          weight : CudaTensor - size: 4096x4096
          train : false
          b

 gradInput : CudaTensor - empty
          gradWeight : CudaTensor - size: 4096x4096
          output : CudaTensor - size: 4096
        }
      37 : 
        cudnn.ReLU
        {
          inplace : true
          mode : CUDNN_ACTIVATION_RELU
          train : false
          output : CudaTensor - size: 4096
          gradInput : CudaTensor - empty
          nElem : 4096
          iDesc : cdata<struct cudnnTensorStruct *[1]>: 0x41fa2e28
        }
      38 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : CudaTensor - empty
          train : false
          p : 0.5
          gradInput : CudaTensor - empty
          output : CudaTensor - size: 4096
        }
      39 : 
        nn.Linear(4096 -> 1000)
        {
          gradBias : CudaTensor - size: 1000
          weight : CudaTensor - size: 1000x4096
          train : false
          bias : CudaTensor - size: 1000
          gradInput : CudaTensor - empty
          gradWeight : CudaTensor - size: 1000x4096
 

 40 : 
        nn.SoftMax
        {
          gradInput : CudaTensor - empty
          train : false
          output : CudaTensor - size: 1000
        }
    }
  train : false
  output : CudaTensor - size: 1000
}
